In [1]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [2]:
mndata = MNIST('../HW1/Dataset')
trainX = np.array(mndata.load_training()[0])[:50000]
trainY = np.array(mndata.load_training()[1])[:50000]

testX = np.array(mndata.load_testing()[0])[:1000]
testY = np.array(mndata.load_testing()[1])[:1000]

In [3]:
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

def feat(data,i):
    return data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [4]:
trnY[0:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [19]:
def sigmoid(x):
    return 1.0/(1+np.exp(-1*np.array(x)))

def grad(x):
    return x*(1-x)

def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x
    
maxValAcc = 0
testAcc = 0
lr = 0.00001

# randomly initialize our weights with mean 0
n_hid_1 = 50
n_hid_2 = 80
W1 = 2*np.random.random((784,n_hid_1)) - 1
bias1 = 2*np.random.random((n_hid_1)) - 1
W2 = 2*np.random.random((n_hid_1,n_hid_2)) - 1
bias2 = 2*np.random.random((n_hid_2)) - 1
W3 = 2*np.random.random((n_hid_2,10)) - 1
bias3 = 2*np.random.random((10)) - 1

for j in xrange(100):
    A1 = np.dot(trnX, W1) + bias1
    l1 = sigmoid(A1)
    
    A2 = np.dot(l1, W2) + bias2
    l2 = sigmoid(A2)
    
    A3 = np.dot(l2, W3) + bias3
    l3 = softmax(A3)

    # Errors in output layer
    d3 = (l3 - trnY)
    dbias3 = np.sum(d3, axis = 0)
        
    # Delta of W3
    dW3 = np.dot(l2.T, d3)

    # Errors in 2nd hidden layer
    d2 = np.dot(d3, W3.T)*grad(l2)
    dbias2 = np.sum(d2, axis = 0)    
    
    # Delta W2
    dW2 = np.dot(l1.T, d2)
    
    # Errors in 2nd hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias1 = np.sum(d1, axis = 0)    
    
    # Delta W3
    dW1 = np.dot(trnX.T, d1)
    
    W3 -= lr*dW3
    bias3 -= lr*dbias3
    W2 -= lr*dW2
    bias2 -= lr*dbias2
    W1 -= lr*dW1
    bias1 -= lr*dbias1
    
    prediction = softmax(np.dot(sigmoid(np.dot(sigmoid(np.dot(valX, W1)+bias1), W2)+bias2), W3)+bias3)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(valY, axis = 1), np.argmax(prediction, axis = 1))]
    valAcc = np.sum(correct)*100.0/len(valX)
    if(valAcc > maxValAcc):
        prediction = softmax(np.dot(sigmoid(np.dot(sigmoid(np.dot(tstX, W1)+bias1), W2)+bias2), W3)+bias3)
        correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(tstY, axis = 1), np.argmax(prediction, axis = 1))]
        testAcc = np.sum(correct)*100.0/len(tstX)
        print testAcc

8.8
12.0
12.8
15.1
15.35
18.2
18.95
21.35
22.8
25.35
26.85
28.95
30.75
32.9
34.35
35.6
37.65
39.05
40.8
41.85
43.0
43.85
44.5
45.8
46.7
47.6
48.25
48.9
49.55
50.8
52.1
53.0
53.75
54.7
55.3
56.15
56.9
57.85
58.4
59.0
59.5
60.2
60.95
61.75
62.05
62.55
63.1
63.55
63.95
64.1
64.25
64.6
65.1
65.45
65.7
65.9
66.3
66.8
67.1
67.25
67.5
67.75
67.95
68.1
68.25
68.5
68.8
69.1
69.25
69.5
69.6
69.9
70.15
70.4
70.75
70.9
71.25
71.3
71.6
71.8
72.15
72.35
72.4
72.65
72.65
72.7
72.9
73.05
73.05
73.1
73.1
73.35
73.45
73.6
73.65
73.6
73.75
73.8
73.95
74.0
